# Granovetter Threshold Model

In [ ]:
# loading igraph library
library(igraph)

We have had a look at how we can use algorithms to build psychological
mechanisms of social network formation into our models.

lets now have a look at how we can model the spread of information, norms,
decisions or opinions within social networks.

The most basic form of model for the spread of such information in networks is the Granovetter Threshold Model.
Its basic idea is that each node in the network starts with one of x different attributes.  In the social sciences
these attributes can represents norms, values, behaviors, opinions or beliefs. After the initialization of the model, the model goes through
different rounds. In each round, each node polls its immediate neighbours and checks the proportion of neighbours that
have a specific attribute. If the proportion of neighbours exceeds a specfic threshold t (e.g. 50%), the node will adopt the
attribute as well.

As such, the model simulates a process of majority influence.

In [ ]:
# lets build a very basic network to start with.
Sender <- c("Bob",
            "Hank",
            "Mat",
            "Fred",
            "Homer",
            "Hank")

Nominees <- c("Hank",
              "Mat",
              "Homer",
              "Bob",
              "Fred",
              "Fred")

# building a graph object
network2 <- data.frame(Sender,Nominees)
net2 <- graph.data.frame(network2, directed=F)

#plotting
set.seed(123)
plot(net2, vertex.size = 30)

Now lets assign the vertices attributes, for example the norms they hold on a specific issue.


Lets say we ask them if they think people should wear suits to work at university.

In [ ]:
# creating a norm attribute as an example
V(net2)
V(net2)$norm <- c("suits","suits","casual","casual","suits")
V(net2)$norm

Lets define another vector to assign colors to the different norms

In [ ]:
# suits = red
# casual = blue

V(net2)$color[V(net2)$norm == "suits"] <- "red"
V(net2)$color[V(net2)$norm == "casual"] <- "blue"

In [ ]:
# lets plot the network
set.seed(123)
plot(net2,
     vertex.color = V(net2)$color,
     vertex.label.color = "white",
     vertex.size = 30,
     main = "Norm of wearing suits or casual clothing to work at University")

# and for better understanding, lets add a legend
legend("bottomleft",
       legend = c("suits","casual"),
       col = c("red","blue"),
       pch = 15,
       bty = "n",
       pt.cex = 3.5,
       cex = 1.8 , 
       text.col = "black",
       horiz = FALSE)

We now have a graph that displays the structure of the network for five people
and the respective norms they hold.

Hank, Homer and Bob think people should wear suits while Mat and Fred think people should dress casually

Lets say the norm evolves in the network according to a Granovetter threshold Model with t = 0.5, meaning
that people will adopt the norm that over 50% of their contacts are displaying

As an example, lets check what will happen to Hank in the Granovetter threshold model

In [ ]:
# First we need to consider Hank and his neighbors
HankEgoNet <- make_ego_graph(net2, 1, V(net2))[[2]]

plot(HankEgoNet,
     vertex.color = V(HankEgoNet)$color,
     vertex.label.color = "white",
     vertex.size = 30,
     main = "Ego Graph for Hank")

legend("bottomleft",
       legend = c("suits","casual"),
       col = c("red","blue"),
       pch = 15,
       bty = "n",
       pt.cex = 3.5,
       cex = 1.8 , 
       text.col = "black",
       horiz = FALSE)

We can see that Hank has 3 neighbors: Mat, Fred and Bob

Bob is suited up while Fred and Mat are dressing casually


Of all of Hanks connections, 1/3 is for suits and 2/3 are for casual clothes.

If we set the threshold to 0.5, Hank should switch to casual clothes because 2/3 > 0.5

lets try to put that logic into code:

In [ ]:
# lets set the threshold
t  <- 0.5

In [ ]:
# First, we need to get the names of all neighbors of Hank:
neighbors <- unlist(adjacent_vertices(net2,2, mode = "all"))
neighbors <- strsplit(names(neighbors),".",fixed=TRUE)
neighbors <- sapply(neighbors,`[`,2)
neighbors 

In [ ]:
# Then we need to check the attributes of all of the neighbors
NeighborGraph <- subgraph(net2,neighbors)

set.seed(123)
plot(NeighborGraph,
     vertex.color = V(NeighborGraph)$color,
     vertex.label.color = "white",
     vertex.size = 30,
     main = "Graph of all neighbors of Hank except for Hank")

V(NeighborGraph)$norm

We need to check the norms that Hanks neighbors are displaying to him

In [ ]:
# checking the norms of Hanks neighbors and converting them into a sorted proportion table
V(NeighborGraph)$norm
PropTable <- sort(prop.table(table(V(NeighborGraph)$norm)), decreasing = TRUE)
PropTable

We can ignore all proportions that are below the threshold value

In [ ]:
# We can discard all elements below the threshold value
PropTable <- PropTable[PropTable > t]
PropTable

In [ ]:
# Picking the attribute with the highest proportion from those that are above the treshold
HanksNewNorm <- names(PropTable[1])
HanksNewNorm

We can see that, as desired, this code would lead to Hank adopting the norm of casual clothing

Lets build a procedure that generalizes this mechanism to the whole network



We need a function that updates the norm for a single node and then apply that functon to all nodes.

In [ ]:
UpdateNorm <- function (node,t = 0.5){
        
        # we switch off warning because R prints a useless warning for using subgraph()
        options(warn=-1)
        
        # getting a vector of all neighbors of the node
        neighbors <- unlist(adjacent_vertices(net2,node, mode = "all"))
        neighbors <- strsplit(names(neighbors),".",fixed=TRUE)
        neighbors <- sapply(neighbors,`[`,2) 
        
        # building a graph object out of neighboring nodes only
        NeighborGraph <- subgraph(net2,neighbors)
        
        # creating a sorted proportion table for their norm attributes
        PropTable <- sort(prop.table(table(V(NeighborGraph)$norm)), decreasing = TRUE)
        
        # deleting these attributes whose proportion is not above the threshold
        PropTable <- PropTable[PropTable > t]
        
        # If no attributes are above the threshold, we do nothing, if yes, then we adopt the remaining
        # attribute with the largest proportion among the nodes' neighbors
        if(length(PropTable) == 0){
                
                NewNorm <- V(net2)[node]$norm
                
        } else if(length(PropTable) > 0) {
                
                NewNorm <- names(PropTable[1])
        }
        
        # we switch the warnings back on again
        options(warn=0)
            
            
        #return the vector with the updated norms
        return(NewNorm)
}

Now we can update all nodes simultanously according to the Granovetter threshold Model

In [ ]:
# Applying the function to all Nodes
NewNorms <- sapply(names(V(net2)),UpdateNorm)
NewNorms

lets compare the updated norms to the initial network

In [ ]:
# plot the network
set.seed(123)
plot(net2,
     vertex.color = V(net2)$color,
     vertex.label.color = "white",
     vertex.size = 30,
     main = "Initial Norm of wearing suits or casual clothing to work at University")

# and for better understanding, lets add a legend
legend("bottomleft",
       legend = c("suits","casual"),
       col = c("red","blue"),
       pch = 15,
       bty = "n",
       pt.cex = 3.5,
       cex = 1.8 , 
       text.col = "black",
       horiz = FALSE)

We now need a function that takes a network as input and then updates all nodes and returns the updated network

In [ ]:
# function for updateing the norms of a whole network with variable threshold parameter t
GranoThreshRound <- function(network,t = 0.5) {
        
        # assigning t to the variable threshold so we can pass it down to the update function
        threshold <- t
        
        # The function for updateing norms (t gets passed down from the GranoThreshRound function)
        UpdateNorm <- function (node,threshold = 0.5){
                
                # we switch off warning because R prints a useless warning for using subgraph()
                options(warn=-1)
                
                # getting a vector of all neighbors of the node
                neighbors <- unlist(adjacent_vertices(network,node, mode = "all"))
                neighbors <- strsplit(names(neighbors),".",fixed=TRUE)
                neighbors <- sapply(neighbors,`[`,2) 
                
                # building a rgaph bject of only the neighbors of the node
                NeighborGraph <- subgraph(network,neighbors)
                
                # creating a sorted proportion table for their norm attributes
                PropTable <- sort(prop.table(table(V(NeighborGraph)$norm)), decreasing = TRUE)
                
                # deleting these attributes whose proportion is not above the threshold
                PropTable <- PropTable[PropTable > threshold]
                
                # If no attributes are above the threshold, we do nothing, if yes, then we adopt the remaining
                # attribute with the largest proportion among the nodes neighbors
                if(length(PropTable) == 0){
                        
                        NewNorm <- V(network)[node]$norm
                        
                } else if(length(PropTable) > 0) {
                        
                        NewNorm <- names(PropTable[1])
                }
                
                # we switch the warnings back on again and return the vector with the updated norms
                options(warn=0)
                return(NewNorm)
        }
        
        # Applying the function to all Nodes
        NewNorms <- sapply(names(V(network)),UpdateNorm)
        
        # Updating the norms attributes of the input network
        V(network)$norm <- NewNorms
        
        # Re-evaluating color attributes for the network
        V(network)$color[V(network)$norm == "suits"] <- "red"
        V(network)$color[V(network)$norm == "casual"] <- "blue"
        
        #returning the nework with updated norms attributes
        return(network)
        
}

lets use the function to do one iteration of the Granovetter Threshold Model

In [ ]:
# applying the function
net2AfterRound1 <- GranoThreshRound(net2)
net2AfterRound1

# inspecting update norms
V(net2AfterRound1)
V(net2AfterRound1)$norm

Lets see in plots what happend after 1 iteration of the Threshold Model

In [ ]:
# setting graphic parameter to display two graphs next to each other
par(mfrow=c(1,2))

# original network
set.seed(123)
plot(net2,
     vertex.color = V(net2)$color,
     vertex.label.color = "white",
     vertex.size = 35,
     main = "Original Network")

legend("bottomleft",
       legend = c("suits","casual"),
       col = c("red","blue"),
       pch = 15,
       bty = "n",
       pt.cex = 3.5,
       cex = 1.8 , 
       text.col = "black",
       horiz = FALSE)

# Network after one iteration
set.seed(123)
plot(net2AfterRound1,
     vertex.color = V(net2AfterRound1)$color,
     vertex.label.color = "white",
     vertex.size = 35,
     main = "After one iteration")

legend("bottomleft",
       legend = c("suits","casual"),
       col = c("red","blue"),
       pch = 15,
       bty = "n",
       pt.cex = 3.5,
       cex = 1.8 , 
       text.col = "black",
       horiz = FALSE)

# reverting graphic parameters
par(mfrow = c(1,1))

lets try with a more complex graph

In [ ]:
# building graph

Sender <- c("Bob",
            "Hank",
            "Mat",
            "Fred",
            "Homer",
            "Hank",
            "Ed",
            "Sergej",
            "Daniel",
            "Daniel",
            "Daniel",
            "Ed",
            "Sergej")

Nominees <- c("Hank",
              "Mat",
              "Homer",
              "Bob",
              "Fred",
              "Fred",
              "Hank",
              "Fred",
              "Bob",
              "Sergej",
              "Ed",
              "Mat",
              "Homer")

# building graph object
network3 <- data.frame(Sender,Nominees)
net3 <- graph.data.frame(network3, directed=F)

plot for overview

In [ ]:
# plot
set.seed(3)
plot(net3, vertex.size = 30)

In [ ]:
# Lets assign norms again
V(net3)$norm <- c("suits","suits","casual","casual","suits","suits","casual","suits")

In [ ]:
# translate it into color attributes
V(net3)$color[V(net3)$norm == "suits"] <- "red"
V(net3)$color[V(net3)$norm == "casual"] <- "blue"

In [ ]:
# and plot the graph with norms
set.seed(3)
plot(net3,
     vertex.color = V(net3)$color,
     vertex.label.color = "white",
     vertex.size = 35,
     main = "More complex network for spreading norms")

legend("bottomleft",
       legend = c("suits","casual"),
       col = c("red","blue"),
       pch = 15,
       bty = "n",
       pt.cex = 3.5,
       cex = 1.8 , 
       text.col = "black",
       horiz = FALSE)

Lets apply the Threshold Model until the network converges onto a single norm

In [ ]:
# Lets apply the Threshold model here
net3AfterRound1 <- GranoThreshRound(net3)

V(net3AfterRound1)$norm
V(net3AfterRound1)$color

In [ ]:
# and plot it
set.seed(3)
plot(net3AfterRound1,
     vertex.color = V(net3AfterRound1)$color,
     vertex.label.color = "white",
     vertex.size = 35,
     main = "More complex network for spreading norms after 1 iteration")

legend("bottomleft",
       legend = c("suits","casual"),
       col = c("red","blue"),
       pch = 15,
       bty = "n",
       pt.cex = 3.5,
       cex = 1.8 , 
       text.col = "black",
       horiz = FALSE)

We can see that Sergej, Matt and Fred changed their norm to wearing suits, while Homer changed from wearing suits to
casual clothing

Lets do another round!

In [ ]:
# Threshold Model
net3AfterRound2 <- GranoThreshRound(net3AfterRound1)

V(net3AfterRound2)$norm
V(net3AfterRound2)$color

# and plot it
set.seed(3)
plot(net3AfterRound2,
     vertex.color = V(net3AfterRound2)$color,
     vertex.label.color = "white",
     vertex.size = 35,
     main = "More complex network for spreading norms after 2 iterations")

legend("bottomleft",
       legend = c("suits","casual"),
       col = c("red","blue"),
       pch = 15,
       bty = "n",
       pt.cex = 3.5,
       cex = 1.8 , 
       text.col = "black",
       horiz = FALSE)

In the next iteration, we can see that Homer switches back again from wearing casual to wearing suits.

The whole network now converged on the norm of wearing suits.

Lets plot all iterations until the convergence in the same plot

In [ ]:
# Lets plot all iterations in one plot to see the convergence on the suits

# setting plotting parameters
par(mfrow = c(1,3))

# original graph
set.seed(3)
plot(net3,
     vertex.color = V(net3)$color,
     vertex.label.color = "white",
     vertex.size = 35,
     main = "Original")

legend("bottomleft",
       legend = c("suits","casual"),
       col = c("red","blue"),
       pch = 15,
       bty = "n",
       pt.cex = 3.5,
       cex = 1.8 , 
       text.col = "black",
       horiz = FALSE)

# after 1 iteration
set.seed(3)
plot(net3AfterRound1,
     vertex.color = V(net3AfterRound1)$color,
     vertex.label.color = "white",
     vertex.size = 35,
     main = "After 1 Iteration")

legend("bottomleft",
       legend = c("suits","casual"),
       col = c("red","blue"),
       pch = 15,
       bty = "n",
       pt.cex = 3.5,
       cex = 1.8 , 
       text.col = "black",
       horiz = FALSE)

# after 2 iterations
set.seed(3)
plot(net3AfterRound2,
     vertex.color = V(net3AfterRound2)$color,
     vertex.label.color = "white",
     vertex.size = 35,
     main = "After 2 Iterations")

legend("bottomleft",
       legend = c("suits","casual"),
       col = c("red","blue"),
       pch = 15,
       bty = "n",
       pt.cex = 3.5,
       cex = 1.8 , 
       text.col = "black",
       horiz = FALSE)